# Prepare and clean data for analysis
---
### Data sources:
- Head et al (2010) "The Erosion of Trade Linkages After Independence" (https://drive.google.com/file/d/15TVAP7PdBV4qatGGOslA-gdGl8AK3EtN/view)
- PolityV Annual Time Series 1946-2018 (https://www.systemicpeace.org/inscrdata.html)

In [ ]:
import io
import numpy as np
import pandas as pd
import tempfile
import requests
import shutil
import xlrd
import zipfile

### 1. Trade flows
---

In [ ]:
tmpdir = tempfile.mkdtemp()
r = requests.get("https://drive.google.com/u/0/uc?id=15TVAP7PdBV4qatGGOslA-gdGl8AK3EtN&export=download")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(tmpdir)
trade_df = pd.read_stata(f"{tmpdir}/col_regfile09.dta")
shutil.rmtree(tmpdir)

In [ ]:
# Only keep years >= 1990
trade_df = trade_df.query('year >= 1990').reset_index(drop=True)
trade_df

In [ ]:
trade_df.columns

### 2. Polity Scores
---

In [ ]:
polity_scores_df = pd.read_excel("http://www.systemicpeace.org/inscr/p5v2018.xls")

# Only keep years >= 1990

polity_scores_df = polity_scores_df.query('year >= 1990').reset_index(drop=True)
polity_scores_df

In [ ]:
polity_scores_df.columns

In [ ]:
# Only keep revised polity score, country, and year
polity_scores_df = polity_scores_df[['country', 'year', 'polity2']]

### 3. ISO country codes
---

In [ ]:
# Get country, ISO3166-1-Alpha-3 dataset

iso_country_df = pd.read_csv("https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv")
iso_country_df = iso_country_df[['CLDR display name', 'ISO3166-1-Alpha-3']]
iso_country_df.columns = ['country', 'iso']

# Rename countries to match PolityIV country names

iso_country_df = iso_country_df.replace('UK', 'United Kingdom')
iso_country_df = iso_country_df.replace('US', 'United States')
iso_country_df = iso_country_df.replace('South Korea', 'Korea South')
iso_country_df = iso_country_df.replace('North Korea', 'Korea North')
iso_country_df = iso_country_df.replace('Czechia', 'Czech Republic')
iso_country_df = iso_country_df.replace('Slovakia', 'Slovak Republic')
iso_country_df = iso_country_df.replace('North Macedonia', 'Yugoslavia')
iso_country_df = iso_country_df.replace('United Arab Emirates', 'UAE')
iso_country_df = iso_country_df.replace('Myanmar', 'Myanmar (Burma)')

# Replace Romanian ISO code

iso_country_df = iso_country_df.replace('ROU', 'ROM')

# Data Preparation
---

In [ ]:
# Merge trade_df and iso_country_df on iso_o / iso_d

trade_df = pd.merge(trade_df, iso_country_df, how='inner', left_on='iso_o', right_on='iso')
trade_df = pd.merge(trade_df, iso_country_df, how='inner', left_on='iso_d', right_on='iso')
trade_df = trade_df.rename(columns={'country_x': 'country_o', 'country_y': 'country_d'})
trade_df = trade_df.drop(columns=['iso_x', 'iso_y'])
trade_df

In [ ]:
# Create new column with trade link (iso_d + iso_d)
trade_df["tradelink"] = trade_df.apply(lambda row: row['iso_o'] + '-' + row['iso_d'], axis=1)
trade_df

In [ ]:
trade_df = pd.merge(left=trade_df, right=polity_scores_df, left_on=['year','country_o'], right_on=['year','country'])
trade_df = pd.merge(left=trade_df, right=polity_scores_df, left_on=['year','country_d'], right_on=['year','country'])
trade_df = trade_df.rename(columns={'polity2_x':'polity_o', 'polity2_y':'polity_d'})
trade_df

In [ ]:
trade_df = trade_df.drop(columns=['country_x', 'country_y'])

In [ ]:
same_countries = sorted(list(set(trade_df['country_o'].unique()) & set(polity_scores_df['country'].unique())))
missing = pd.DataFrame(set(polity_scores_df['country'].unique()).difference(set(same_countries)))
missing.columns = ['Missing Countries']
missing

In [ ]:
# Append difference in scores column to trade_df
trade_df['polity_dist'] = abs(trade_df['polity_o'] - trade_df['polity_d'])
trade_df

In [ ]:
trade_df.to_csv("../data/trade-polity.csv")